## Setup

In [1]:
import json

import fiftyone as fo
from fiftyone import ViewField as F

In [2]:
# A name for the dataset
name = "my-dataset"

# The directory containing the source images
data_path = "/media/data/datasets/TIL2021_CV_dataset/images/c3_test/images"

# The path to the COCO labels JSON file
labels_path = "/media/data/datasets/TIL2021_CV_dataset/images/c3_test/c3_test.json"

# The path to the COCO predictions JSON file
preds_path = "/media/data/datasets/TIL2021_CV_dataset/images/c3_test/til21_c3_pred.json"

In [3]:
with open(labels_path) as json_file:
    labels_data = json.load(json_file)
    
with open(preds_path) as json_file:
    preds_data = json.load(json_file)

In [4]:
# Import the dataset
dataset = fo.Dataset.from_dir(
    dataset_type=fo.types.COCODetectionDataset,
    data_path=data_path,
    labels_path=labels_path,
    name=name,
)

 100% |█████████████████| 400/400 [830.9ms elapsed, 0s remaining, 482.8 samples/s]      


In [5]:
dataset

Name:        my-dataset
Media type:  image
Num samples: 400
Persistent:  False
Tags:        []
Sample fields:
    id:           fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)

In [9]:
# Print a sample from the dataset
sample = dataset.first()
sample

<Sample: {
    'id': '60e323641ac6220b1cbff789',
    'media_type': 'image',
    'filepath': '/media/data/datasets/TIL2021_CV_dataset/images/c3_test/images/1.jpg',
    'tags': BaseList([]),
    'metadata': <ImageMetadata: {
        'size_bytes': None,
        'mime_type': None,
        'width': 1024,
        'height': 768,
        'num_channels': None,
    }>,
    'ground_truth': <Detections: {
        'detections': BaseList([
            <Detection: {
                'id': '60e323641ac6220b1cbff783',
                'attributes': BaseDict({}),
                'tags': BaseList([]),
                'label': 'Chicken',
                'bounding_box': BaseList([
                    0.73994140625,
                    0.38212239583333335,
                    0.26005859375,
                    0.48194010416666666,
                ]),
                'mask': None,
                'confidence': None,
                'index': None,
                'supercategory': '',
                'iscrowd': 

In [10]:
# Print a ground truth detection
sample.ground_truth.detections[0]

<Detection: {
    'id': '60e323641ac6220b1cbff783',
    'attributes': BaseDict({}),
    'tags': BaseList([]),
    'label': 'Chicken',
    'bounding_box': BaseList([
        0.73994140625,
        0.38212239583333335,
        0.26005859375,
        0.48194010416666666,
    ]),
    'mask': None,
    'confidence': None,
    'index': None,
    'supercategory': '',
    'iscrowd': 0,
}>

In [7]:
# Launch the App in a dedicated browser tab
session = fo.launch_app(dataset, auto=False)
session.open_tab()

Session launched. Run `session.show()` to open the App in a cell output.


## Add predictions to dataset

In [ ]:
# Add predictions to samples
cat2name = {cat['id']:cat['name'] for cat in labels_data["categories"]}

with fo.ProgressBar() as pb:
    for sample in pb(dataset):
        filepath = sample.filepath
        filename = filepath.split('/')[-1]

        # get image_id of sample
        img_data = next(item for item in labels_data["images"] if item["file_name"] == filename)
        image_id = img_data["id"]
        img_w = img_data["width"]
        img_h = img_data["height"]

        # Convert detections to FiftyOne format
        detections = []
        for pred in (x for x in preds_data if x["image_id"] == image_id):
            # Convert to [top-left-x, top-left-y, width, height] in relative coordinates in [0, 1] x [0, 1]
            x, y, w, h = pred["bbox"]
            rel_box = [x / img_w, y / img_h, w / img_w, h / img_h]

            detections.append(
                fo.Detection(
                    label=cat2name[pred["category_id"]],
                    bounding_box=rel_box,
                    confidence=pred["score"]
                )
            )

        # Save predictions to dataset
        sample["predictions"] = fo.Detections(detections=detections)
        sample.save()

## Analyzing detections

### Evaluate detections

In [ ]:
# Evaluate the predictions of our dataset with respect to the objects in the `ground_truth` field
all_results = dataset.evaluate_detections(
    pred_field="predictions",
    gt_field="ground_truth",
    eval_key="eval",
    compute_mAP=True,
)

In [ ]:
# Compute the mean average-precision (mAP) of our detector
print(all_results.mAP())

In [ ]:
# View some precision-recall (PR) curves for specific classes of our model 
plot = all_results.plot_pr_curves(classes=dataset.distinct("ground_truth.detections.label"))
plot.show()

### Confidence thresholding

In [ ]:
# Only keep detections with confidence >= 0.5
high_conf_view = dataset.filter_labels("predictions", F("confidence") > 0.5)
session.view = high_conf_view

In [ ]:
# Evaluate the predictions of 'high_conf_view' with respect to the objects in the `ground_truth` field
high_conf_results = high_conf_view.evaluate_detections(
    pred_field="predictions",
    gt_field="ground_truth",
    eval_key="eval_high_conf",
)

# Print a classification report
high_conf_results.print_report()

In [ ]:
# In order for the confusion matrix to capture anything other than false positive/negative counts,
# you will likely want to set the classwise parameter to False during evaluation so that detections 
# can be matched with ground truth objects of different classes.

# Perform evaluation, allowing objects to be matched between classes
conf_matrix_results = high_conf_view.evaluate_detections(
    pred_field="predictions",
    gt_field="ground_truth",
    classwise=False
)

# Generate a confusion matrix
plot = conf_matrix_results.plot_confusion_matrix(classes=dataset.distinct("ground_truth.detections.label"))
plot.show()

### View the best-performing samples

In [ ]:
# Show samples with most true positives
session.view = high_conf_view.sort_by("eval_tp", reverse=True)

### View the worst-performing samples

In [ ]:
# Show samples with most false positives
session.view = high_conf_view.sort_by("eval_fp", reverse=True)

### Filtering by bounding box area

In [ ]:
# Evaluate only certain sized objects in a dataset
dataset.compute_metadata()

# Create an expression that will match objects by bounding box area
bbox_area = (
    F("$metadata.width") * F("bounding_box")[2] *
    F("$metadata.height") * F("bounding_box")[3]
)

In [ ]:
# sized_boxes = bbox_area < 32 ** 2 # small boxes: area smaller than 32^2 pixels
# sized_boxes = (32 ** 2 < bbox_area) & (bbox_area < 96 ** 2) # medium boxes: area between 32^2 and 96^2 pixels
sized_boxes = bbox_area > 96 ** 2 # large boxes: area larger than 96^2 pixels

In [ ]:
# Create a view that contains only certain sized (and high confidence) predictions
sized_boxes_view = (
    high_conf_view
    .filter_labels("ground_truth", sized_boxes)
    .filter_labels("predictions", sized_boxes)
)

session.view = sized_boxes_view

In [ ]:
# Evaluate only certain sized objects
sized_boxes_results = sized_boxes_view.evaluate_detections(
    pred_field="predictions",
    gt_field="ground_truth",
    eval_key="eval_sized",
)

# Print a classification report
sized_boxes_results.print_report()

## Tagging the predictions with missing ground truth detections

In [ ]:
# Tag all highly confident false positives as "possibly-missing"
(
    high_conf_view
        .filter_labels("predictions", F("eval") == "fp")
        .select_fields("predictions")
        .tag_labels("possibly-missing")
)

In [ ]:
# Export all labels with the `possibly-missing` tag in COCO format
export_path = "probably_missing"
(
    dataset
        .select_labels(tags=["possibly-missing"])
        .export(export_path, fo.types.COCODetectionDataset)
)